## Demonstrate interactions with relations using omnikeeper

In [137]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import pandas as pd

As usual create the omnikeeper client

In [138]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

In this example we will work with two traits host and interface and we will define relations using okc public functions. 

First lets define the trait names and layer.

In [139]:
host_trait_name = "python_client_demo.host"
interface_trait_name = "python_client_demo.interface"
layer_name = "relations"

Make sure that traits are created correctly

In [140]:
is_host_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(host_trait_name, [
        TraitAttributeDefinition("id", "host.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_interface", predicate_id = "has_interface", direction_forward=True)
      ]))
print(is_host_trait_created)

is_interface_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(interface_trait_name, [
        TraitAttributeDefinition("id", "interface.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "attached_to_host", predicate_id = "has_interface", direction_forward=False)
      ]))
print(is_interface_trait_created)

True
True


Create the host and interface CIs

Create host traitentities

In [141]:
hosts_list = [
    {"id": 1},
    {"id": 2},
    {"id": 3},
]
okc.bulk_replace_trait_entities_by_filter_list(okapiclient, trait_name=host_trait_name, input=hosts_list, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

True

Create Interface traitentities

In [142]:
interfaces_list = [
    {"id": 1},
    {"id": 2}, 
    {"id": 3}
]
okc.bulk_replace_trait_entities_by_filter_list(okapiclient, trait_name=interface_trait_name, input=interfaces_list, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

True

Fetch the created trait entities

In [143]:
hosts = okc.get_all_traitentities_list(okapiclient, trait_name=host_trait_name, layers=[layer_name])
hosts

[{'ciid': '4cf115e2-ba43-4fd6-a8ce-969e6c536523', 'id': 1},
 {'ciid': 'c1c01eb1-2883-4a8e-bd9d-645b68ed06fe', 'id': 2},
 {'ciid': '713dcb55-f47c-48df-8461-16faf3444598', 'id': 3}]

In [144]:
interfaces = okc.get_all_traitentities_list(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
interfaces

[{'ciid': '1ec74667-2ebb-493c-87d1-8493b21c4f03', 'id': 1},
 {'ciid': 'e702709b-dbc1-482c-8bd0-9db5925fd870', 'id': 2},
 {'ciid': '624d8bb8-3ea5-4573-98d0-3f9335357bd2', 'id': 3}]

Add host and interface relations using the created cis and okc

In [145]:
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", hosts[0]["ciid"], [interfaces[0]["ciid"], interfaces[1]["ciid"]], layer_name)
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", hosts[1]["ciid"], [interfaces[2]["ciid"]], layer_name)

True

### Relations operations using lists

Fetch has_host realtions. In the result interfaces will be as base_cis and hosts as related cis

In [146]:
attached_to_host_relations = okc.get_trait_relation_list(okapiclient, trait_name=interface_trait_name, relation_name="attached_to_host", layers=[layer_name])
attached_to_host_relations

[{'base_ciid': '1ec74667-2ebb-493c-87d1-8493b21c4f03',
  'related_ciids': ['4cf115e2-ba43-4fd6-a8ce-969e6c536523']},
 {'base_ciid': 'e702709b-dbc1-482c-8bd0-9db5925fd870',
  'related_ciids': ['4cf115e2-ba43-4fd6-a8ce-969e6c536523']},
 {'base_ciid': '624d8bb8-3ea5-4573-98d0-3f9335357bd2',
  'related_ciids': ['c1c01eb1-2883-4a8e-bd9d-645b68ed06fe']}]

Fetch has_interface relations for all hosts

In [147]:
has_interface_relations = okc.get_trait_relation_list(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations

[{'base_ciid': '4cf115e2-ba43-4fd6-a8ce-969e6c536523',
  'related_ciids': ['1ec74667-2ebb-493c-87d1-8493b21c4f03',
   'e702709b-dbc1-482c-8bd0-9db5925fd870']},
 {'base_ciid': 'c1c01eb1-2883-4a8e-bd9d-645b68ed06fe',
  'related_ciids': ['624d8bb8-3ea5-4573-98d0-3f9335357bd2']},
 {'base_ciid': '713dcb55-f47c-48df-8461-16faf3444598', 'related_ciids': []}]

Merge hosts with realted cis in this case interfaces

In [148]:
interface_dict = {interface['ciid']: interface for interface in interfaces}

hosts_with_relations = []
for host in hosts:
    merged_dict = host.copy()
    merged_dict['related_ciis'] = []
    for relation in has_interface_relations:
        if host['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in interface_dict:
                    merged_dict['related_ciis'].append(interface_dict[related_ciid])
    hosts_with_relations.append(merged_dict)

hosts_with_relations

[{'ciid': '4cf115e2-ba43-4fd6-a8ce-969e6c536523',
  'id': 1,
  'related_ciis': [{'ciid': '1ec74667-2ebb-493c-87d1-8493b21c4f03', 'id': 1},
   {'ciid': 'e702709b-dbc1-482c-8bd0-9db5925fd870', 'id': 2}]},
 {'ciid': 'c1c01eb1-2883-4a8e-bd9d-645b68ed06fe',
  'id': 2,
  'related_ciis': [{'ciid': '624d8bb8-3ea5-4573-98d0-3f9335357bd2', 'id': 3}]},
 {'ciid': '713dcb55-f47c-48df-8461-16faf3444598', 'id': 3, 'related_ciis': []}]

The same can be done for interfaces to fetch its relations

In [149]:
host_dict = {host['ciid']: host for host in hosts}

interfaces_with_relations = []
for interface in interfaces:
    merged_dict = interface.copy()
    merged_dict['related_ciis'] = []
    for relation in attached_to_host_relations:
        if interface['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in host_dict:
                    merged_dict['related_ciis'].append(host_dict[related_ciid])
    interfaces_with_relations.append(merged_dict)

interfaces_with_relations

[{'ciid': '1ec74667-2ebb-493c-87d1-8493b21c4f03',
  'id': 1,
  'related_ciis': [{'ciid': '4cf115e2-ba43-4fd6-a8ce-969e6c536523', 'id': 1}]},
 {'ciid': 'e702709b-dbc1-482c-8bd0-9db5925fd870',
  'id': 2,
  'related_ciis': [{'ciid': '4cf115e2-ba43-4fd6-a8ce-969e6c536523', 'id': 1}]},
 {'ciid': '624d8bb8-3ea5-4573-98d0-3f9335357bd2',
  'id': 3,
  'related_ciis': [{'ciid': 'c1c01eb1-2883-4a8e-bd9d-645b68ed06fe', 'id': 2}]}]

### Relations operations using pandas dataframes

We will do the same steps here but using pandas dataframes to work with omnikeeper data

In [150]:
has_interface_relations_df = okc.get_trait_relation_dataframe(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations_df

,related_ciids
base_ciid,
4cf115e2-ba43-4fd6-a8ce-969e6c536523,"[1ec74667-2ebb-493c-87d1-8493b21c4f03, e702709..."
c1c01eb1-2883-4a8e-bd9d-645b68ed06fe,[624d8bb8-3ea5-4573-98d0-3f9335357bd2]
713dcb55-f47c-48df-8461-16faf3444598,[]


Fetch hosts and interfaces using dataframes 

In [151]:
hosts_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=host_trait_name, layers=[layer_name])
interfaces_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
hosts_df

,id
ciid,
4cf115e2-ba43-4fd6-a8ce-969e6c536523,1
c1c01eb1-2883-4a8e-bd9d-645b68ed06fe,2
713dcb55-f47c-48df-8461-16faf3444598,3


In [152]:
hosts_df.index.name

'ciid'

In [153]:
has_interface_relations_df.index.name

'base_ciid'

Now merge the related data. In this case we can see how easy it is to merge related data using pandas dataframes

In [154]:
hosts_with_relations_df = hosts_df.merge(has_interface_relations_df, left_index=True, right_index=True, how='outer')
hosts_with_relations_df

,id,related_ciids
4cf115e2-ba43-4fd6-a8ce-969e6c536523,1,"[1ec74667-2ebb-493c-87d1-8493b21c4f03, e702709..."
c1c01eb1-2883-4a8e-bd9d-645b68ed06fe,2,[624d8bb8-3ea5-4573-98d0-3f9335357bd2]
713dcb55-f47c-48df-8461-16faf3444598,3,[]


We can create a multi index dataframe which includes interfaces information

In [155]:
df_exploded = hosts_with_relations_df.explode('related_ciids').reset_index()
df_exploded = df_exploded.rename(columns={df_exploded.columns[0]: 'ciid'})
df_exploded

,ciid,id,related_ciids
0,4cf115e2-ba43-4fd6-a8ce-969e6c536523,1,1ec74667-2ebb-493c-87d1-8493b21c4f03
1,4cf115e2-ba43-4fd6-a8ce-969e6c536523,1,e702709b-dbc1-482c-8bd0-9db5925fd870
2,c1c01eb1-2883-4a8e-bd9d-645b68ed06fe,2,624d8bb8-3ea5-4573-98d0-3f9335357bd2
3,713dcb55-f47c-48df-8461-16faf3444598,3,NaN


In [156]:
merged_df = pd.merge(df_exploded, interfaces_df, left_on='related_ciids', right_on='ciid', how='left', suffixes=('_host', '_interface'))
merged_df.set_index(['ciid', 'related_ciids'], inplace=True)
merged_df

id_host  \
ciid                                 related_ciids                                   
4cf115e2-ba43-4fd6-a8ce-969e6c536523 1ec74667-2ebb-493c-87d1-8493b21c4f03        1   
                                     e702709b-dbc1-482c-8bd0-9db5925fd870        1   
c1c01eb1-2883-4a8e-bd9d-645b68ed06fe 624d8bb8-3ea5-4573-98d0-3f9335357bd2        2   
713dcb55-f47c-48df-8461-16faf3444598 NaN                                         3   

                                                                           id_interface  
ciid                                 related_ciids                                       
4cf115e2-ba43-4fd6-a8ce-969e6c536523 1ec74667-2ebb-493c-87d1-8493b21c4f03           1.0  
                                     e702709b-dbc1-482c-8bd0-9db5925fd870           2.0  
c1c01eb1-2883-4a8e-bd9d-645b68ed06fe 624d8bb8-3ea5-4573-98d0-3f9335357bd2           3.0  
713dcb55-f47c-48df-8461-16faf3444598 NaN                                            NaN